In [112]:
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
plt.style.use("dark_background")

In [321]:
"""
Trying out a sample code:
"""


def montecarlo_ss(a, b, n):
    # Timing the process:
    start_time = time.time()
    # An array of x values with n numbers in it that will be tested:
    xs = (b-a) * np.random.rand(n) + a
    # The line below basically the function f(x). We can also define f(x) seperately and call it here
    # but that would make the program (about 4 times) slower.
    ys = np.exp(-xs**2)
    # Integral answer:
    I = (b-a) * np.mean(ys)
    # Statistical error:
    delta = np.sqrt((np.mean(ys**2) - (np.mean(ys))**2) / n)
    stop_time = time.time()
    elapsed_time = stop_time - start_time


    print(f"Calculated I    = {I:.6f}")
    # The real answer for this specific function for the given interval taken from wolframalpha.com:
    print(f"Wolfram Alpha I = 0.882081")
    print(f"Stat Delta      = {delta:.6f}")
    print(f"Real Delta      = {(I - 0.882081):.6f}")
    print(f"Runtime (s)     = {elapsed_time:.6f}")

In [322]:
montecarlo_ss(0, 2, 100_000)

Calculated I    = 0.882904
Wolfram Alpha I = 0.882081
Stat Delta      = 0.001090
Real Delta      = 0.000823
Runtime (s)     = 0.001245


In [323]:
"""
Redefining the function to return (instead of print) the results:
"""


def montecarlo_ss(a, b, n):
    """
    Simple Sampling Monte Carlo for f(x) = exp(-x^2)
    
    a and b: Determine the interval (a, b) in which the integral is to be calculated.

    n: The number of random samples taken out to calculate the integral.
    """

    
    # Timing the process:
    start_time = time.time()
    # An array of x values with n numbers in it that will be tested:
    xs = (b-a) * np.random.rand(n) + a
    # The line below basically the function f(x). We can also define f(x) seperately and call it here
    # but that would make the program (about 4 times) slower.
    ys = np.exp(-xs**2)
    # Integral answer:
    I = (b-a) * np.mean(ys)
    # Statistical error:
    delta = np.sqrt((np.mean(ys**2) - (np.mean(ys))**2) / n)
    stop_time = time.time()
    elapsed_time = stop_time - start_time


    return I, delta, (I - 0.8820814), elapsed_time

In [324]:
"""
Outputting a table of values:
"""


# This cell takes 28 seconds to execute.
# We will run the function montecarlo_ss() for different values of n:
ns = [100, 200, 500, 800, 1000, 10_000, 100_000, 1_000_000, 10_000_000]
# For each value of n, we will run it 100 times and record the average of the results:
ensemble_size = 100
# This will hold the results for each n:
temp = np.zeros(shape=(ensemble_size, 4))
# This is the data for the dataframe, it holds the average results for each n:
data = np.zeros(shape=(len(ns), 4))
for i in range(len(ns)):
    for j in range(ensemble_size):
        temp[j,:] = montecarlo_ss(0, 2, ns[i])
    data[i,:] = np.mean(temp, axis=0)

In [326]:
# Columns for the dataframe:
columns = ["INTEGRAL ANSWER", "STATISTICAL ERROR", "REAL ERROR (checked with wolfram alpha)", "EXECUTION TIME (s)"]
df = pd.DataFrame(data=data,
                  index=ns,
                  columns=columns)
df.index.name = "n"
df

,INTEGRAL ANSWER,STATISTICAL ERROR,REAL ERROR (checked with wolfram alpha),EXECUTION TIME (s)
n,,,,
100,0.875365,0.034186,-0.006716,0.000020
200,0.883775,0.024262,0.001694,0.000030
500,0.881478,0.015389,-0.000603,0.000010
800,0.879015,0.012160,-0.003066,0.000040
1000,0.882791,0.010901,0.000710,0.000030
10000,0.881731,0.003445,-0.000350,0.000121
100000,0.882088,0.001090,0.000006,0.001069
1000000,0.882026,0.000345,-0.000056,0.020180
10000000,0.882115,0.000109,0.000034,0.207601


In [339]:
# df.to_csv("Exercise7_1_SSMC_dataframe.csv")

In [327]:
def montecarlo_is(a, b, n):
    """
    Intelligent Sampling Monte Carlo for f(x) = exp(-x^2) and g(x) = exp(-x)
    
    a and b: Determine the interval (a, b) in which the integral is to be calculated.

    n: The number of random samples taken out to calculate the integral.
    """

    
    # Timing the process:
    start_time = time.time()
    # An array of x values with n numbers in it that will be tested, drawn from exponential distribution.
    # The exponential distribution is created in place from standard uniform distribution.
    xs = - np.log((b-a) * np.random.rand(n) + a)
    # The two lines below are basically the functions f(x) and g(x), respectively.
    # We can also define f(x) and g(x) seperately and call them here
    # but that would make the program (about 4 times) slower.
    # Function calls generally make the code slower;
    # this code is specifically written for the given functions f(x) and g(x),
    # with efficiency being a priority.
    fs = np.exp(-xs**2)
    gs = np.exp(-xs)
    # Integral answer. It is basically equation 7.14 of the textbook: Int(g) * <f/g>:
    I = (1 - np.exp(-2)) * np.mean(fs / gs)
    # Statistical error:
    delta = np.sqrt((np.mean(fs**2) - (np.mean(fs))**2) / n)
    stop_time = time.time()
    elapsed_time = stop_time - start_time


    return I, delta, (I - 0.882081), elapsed_time

In [328]:
"""
Testing montecarlo_is() function:
"""
# Note that the range of the integral is not (0,2) but (exp(-2), 1).
# It was also possible to convert the range within the definition of the function, no significant difference.
montecarlo_is(np.exp(-2), 1, 100_000)

(0.8822536748636861,
 0.0010446003329939468,
 0.0001726748636861064,
 0.003000020980834961)

In [331]:
"""
Outputting a table of values:
"""


# This cell takes 41 seconds to execute.
# We will run the function montecarlo_ss() for different values of n:
ns = [100, 200, 500, 800, 1000, 10_000, 100_000, 1_000_000, 10_000_000]
# For each value of n, we will run it 100 times and record the average of the results:
ensemble_size = 100
# This will hold the results for each n:
temp = np.zeros(shape=(ensemble_size, 4))
# This is the data for the dataframe, it holds the average results for each n:
data2 = np.zeros(shape=(len(ns), 4))
for i in range(len(ns)):
    for j in range(ensemble_size):
        temp[j,:] = montecarlo_is(np.exp(-2), 1, ns[i])
    data2[i,:] = np.mean(temp, axis=0)

In [332]:
# Columns for the dataframe:
columns2 = ["INTEGRAL ANSWER", "STATISTICAL ERROR", "REAL ERROR (checked with wolfram alpha)", "EXECUTION TIME (s)"]
df2 = pd.DataFrame(data=data2,
                  index=ns,
                  columns=columns2)
df2.index.name = "n"
df2

,INTEGRAL ANSWER,STATISTICAL ERROR,REAL ERROR (checked with wolfram alpha),EXECUTION TIME (s)
n,,,,
100,0.887419,0.032468,0.005338,0.000020
200,0.882636,0.023299,0.000555,0.000030
500,0.884130,0.014723,0.002049,0.000045
800,0.881170,0.011698,-0.000911,0.000035
1000,0.883520,0.010423,0.001439,0.000055
10000,0.882035,0.003305,-0.000046,0.000200
100000,0.881942,0.001045,-0.000139,0.002465
1000000,0.882077,0.000330,-0.000004,0.035727
10000000,0.882087,0.000105,0.000006,0.364646


In [340]:
# df.to_csv("Exercise7_1_ISMC_dataframe.csv")